In [1]:
import sys
!{sys.executable} -m pip install openpyxl



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip


In [2]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data/raw")
OUTPUT_DIR = Path("../data/processed")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ============================================================
# 0. TEAM NAME MAPPING
# ============================================================

team_abbr_to_name = {
    "ARI": "Cardinals",   "ATL": "Falcons",    "BAL": "Ravens",
    "BUF": "Bills",       "CAR": "Panthers",   "CHI": "Bears",
    "CIN": "Bengals",     "CLE": "Browns",     "DAL": "Cowboys",
    "DEN": "Broncos",     "DET": "Lions",      "GB": "Packers",
    "HOU": "Texans",      "IND": "Colts",      "JAX": "Jaguars",
    "KC": "Chiefs",       "LAC": "Chargers",   "LAR": "Rams",
    "MIA": "Dolphins",    "MIN": "Vikings",    "NE": "Patriots",
    "NO": "Saints",       "NYG": "Giants",     "NYJ": "Jets",
    "LV": "Raiders",      "OAK": "Raiders",    "PHI": "Eagles",
    "PIT": "Steelers",    "SEA": "Seahawks",   "SF": "49ers",
    "TB": "Buccaneers",   "TEN": "Titans",     "WAS": "Commanders",
    "WSH": "Commanders"
}

def apply_team_name_mapping(df, col="team"):
    df[col] = df[col].astype(str).str.strip().str.upper()
    df["team_name"] = df[col].replace(team_abbr_to_name)
    return df


# ============================================================
# 1. LOAD RAW FILES
# ============================================================

yearly_team_off = pd.read_csv(DATA_DIR / "yearly_team_stats_offense.csv")
yearly_team_def = pd.read_csv(DATA_DIR / "yearly_team_stats_defense.csv")
weekly_team_def = pd.read_csv(DATA_DIR / "weekly_team_stats_defense.csv")
pos_group       = pd.read_excel(DATA_DIR / "NFL Salary By Position Group.xlsx")

# ============================================================
# 1.1 FILTER TO REGULAR SEASON ONLY
# ============================================================

yearly_team_off = yearly_team_off[yearly_team_off["season_type"] == "REG"]
yearly_team_def = yearly_team_def[yearly_team_def["season_type"] == "REG"]
weekly_team_def = weekly_team_def[weekly_team_def["season_type"] == "REG"]

print("Filtered yearly + weekly to REG only.")


# ============================================================
# 2. TEAM NAME NORMALIZATION
# ============================================================

yearly_team_off = apply_team_name_mapping(yearly_team_off, col="team")
yearly_team_def = apply_team_name_mapping(yearly_team_def, col="team")
weekly_team_def = apply_team_name_mapping(weekly_team_def, col="team")

pos_group = pos_group.rename(columns={
    "Team": "team_name",
    "Season": "season"
})

print("Team names normalized.")


# ============================================================
# 3. RENAME OVERLAPPING COLUMNS (NO POINTS HERE)
# ============================================================

off_rename = {
    "fumble": "fumble_off", "fumble_forced": "fumble_forced_off",
    "fumble_lost": "fumble_lost_off", "fumble_not_forced": "fumble_not_forced_off",
    "fumble_out_of_bounds": "fumble_out_of_bounds_off",

    "average_fumble": "average_fumble_off", "average_fumble_forced": "average_fumble_forced_off",
    "average_fumble_lost": "average_fumble_lost_off", "average_fumble_not_forced": "average_fumble_not_forced_off",
    "average_fumble_out_of_bounds": "average_fumble_out_of_bounds_off",

    "interception": "interception_off", "average_interception": "average_interception_off",
    "safety": "safety_off", "average_safety": "average_safety_off",

    "pass_pct": "pass_pct_off", "rush_pct": "rush_pct_off",
    "pass_snaps": "pass_snaps_off", "rush_snaps": "rush_snaps_off",

    "win": "wins", "loss": "losses", "tie": "ties",
    "win_pct": "win_pct", "record": "record",

    "total_off_points": "total_points_scored"
}

def_rename = {
    "fumble": "fumble_def", "fumble_forced": "fumble_forced_def",
    "fumble_lost": "fumble_lost_def", "fumble_not_forced": "fumble_not_forced_def",
    "fumble_out_of_bounds": "fumble_out_of_bounds_def",

    "average_fumble": "average_fumble_def", "average_fumble_forced": "average_fumble_forced_def",
    "average_fumble_lost": "average_fumble_lost_def", "average_fumble_not_forced": "average_fumble_not_forced_def",
    "average_fumble_out_of_bounds": "average_fumble_out_of_bounds_def",

    "interception": "interception_def", "average_interception": "average_interception_def",
    "safety": "safety_def", "average_safety": "average_safety_def",

    "pass_pct": "pass_pct_def", "rush_pct": "rush_pct_def",
    "pass_snaps": "pass_snaps_def", "rush_snaps": "rush_snaps_def",

    "win": "def_win", "loss": "def_loss", "tie": "def_tie",
    "win_pct": "def_win_pct", "record": "def_record",

    # defensive scoring (not allowed points)
    "def_touchdown": "total_def_points_scored"
}

yearly_team_off = yearly_team_off.rename(columns=off_rename)
yearly_team_def = yearly_team_def.rename(columns=def_rename)

yearly_team_def = yearly_team_def.drop(
    columns=["def_win","def_loss","def_tie","def_win_pct","def_record"],
    errors="ignore"
)


# ============================================================
# 4. COMPUTE TOTAL POINTS ALLOWED FROM WEEKLY DEF STATS
# ============================================================

# Debug: ensure column exists
print("\nDEBUG weekly_team_def point columns:",
      [c for c in weekly_team_def.columns if "point" in c.lower()])

# Aggregate: sum total_off_points per team_name + season
weekly_points_allowed = (
    weekly_team_def
    .groupby(["team_name", "season"])["total_off_points"]
    .sum()
    .reset_index()
    .rename(columns={"total_off_points": "total_points_allowed"})
)

print("Weekly points_allowed shape:", weekly_points_allowed.shape)
print(weekly_points_allowed.head())


# ============================================================
# 5. MERGE OFFENSE + DEFENSE
# ============================================================

team_merged = yearly_team_off.merge(
    yearly_team_def,
    on=["team_name", "season"],
    how="inner"
)

print("Merged OFF + DEF:", team_merged.shape)


# ============================================================
# 6. MERGE IN POINTS ALLOWED (FROM WEEKLY DEF)
# ============================================================

team_merged = team_merged.merge(
    weekly_points_allowed,
    on=["team_name", "season"],
    how="left"
)

print("After merging points_allowed:", team_merged.shape)
print("DEBUG has total_points_allowed?:", "total_points_allowed" in team_merged.columns)


# ============================================================
# 7. MERGE POSITIONAL SALARY DATA
# ============================================================

team_merged = team_merged.merge(
    pos_group,
    on=["team_name", "season"],
    how="left"
)

print("Merged salary data:", team_merged.shape)


# ============================================================
# 8. DROP NON-SALARY YEARS
# ============================================================

salary_percent_cols = [
    'QB_P','RB_P','WR_P','TE_P','OL_P',
    'IDL_P','EDGE_P','LB_P','S_P','CB_P',
    'Offense_P','Defense_P'
]

team_merged = team_merged.dropna(subset=salary_percent_cols, how="any")

print("After dropping missing-salary seasons:", team_merged.shape)


# ============================================================
# 9. SAVE + SANITY CHECK
# ============================================================

output_path = OUTPUT_DIR / "team_season_merged_corrected.csv"
team_merged.to_csv(output_path, index=False)

print(f"\nSaved merged dataset to: {output_path}")

print("\n=== HEAD ===")
display(team_merged.head())

print("\n=== POINT-RELATED COLUMNS ===")
print([c for c in team_merged.columns if "point" in c.lower()])

print("\n=== TOP MISSING VALUES ===")
display(team_merged.isna().sum().sort_values(ascending=False).head(20))


Filtered yearly + weekly to REG only.
Team names normalized.

DEBUG weekly_team_def point columns: ['defensive_two_point_attempt', 'defensive_two_point_conv', 'defensive_extra_point_attempt', 'defensive_extra_point_conv', 'total_off_points', 'total_def_points', 'extra_point', 'average_defensive_two_point_attempt', 'average_defensive_two_point_conv', 'average_defensive_extra_point_attempt', 'average_defensive_extra_point_conv']
Weekly points_allowed shape: (416, 3)
  team_name  season  total_points_allowed
0     49ers    2012                 256.0
1     49ers    2013                 274.0
2     49ers    2014                 319.0
3     49ers    2015                 372.0
4     49ers    2016                 470.0
Merged OFF + DEF: (416, 141)
After merging points_allowed: (416, 142)
DEBUG has total_points_allowed?: True
Merged salary data: (416, 173)
After dropping missing-salary seasons: (310, 173)

Saved merged dataset to: ../data/processed/team_season_merged_corrected.csv

=== HEAD ===

,team_x,season,season_type_x,shotgun,no_huddle,qb_dropback,qb_scramble,total_off_yards,pass_attempts,complete_pass,...,OL_P,IDL_P,EDGE_P,LB_P,S_P,CB_P,Defense_P,Offense_P,Dead_Open_Specials,W
32,ARI,2013,REG,415.0,18.0,622.0,6.0,5831,552.0,363.0,...,0.155111,0.165841,0.043266,0.064434,0.022732,0.084206,0.380479,0.405332,0.214190,10.0
33,ATL,2013,REG,651.0,129.0,710.0,8.0,5788,642.0,445.0,...,0.133204,0.079343,0.065852,0.068234,0.058656,0.075050,0.347135,0.469434,0.183431,4.0
34,BAL,2013,REG,850.0,430.0,685.0,18.0,5242,596.0,363.0,...,0.132321,0.139768,0.156315,0.037160,0.030853,0.088586,0.452683,0.352961,0.194356,8.0
35,BUF,2013,REG,764.0,359.0,607.0,42.0,5680,507.0,299.0,...,0.116953,0.134394,0.107362,0.043809,0.084979,0.057199,0.427743,0.331319,0.240938,6.0
36,CAR,2013,REG,633.0,35.0,551.0,40.0,5405,460.0,292.0,...,0.153569,0.041231,0.094093,0.099805,0.053720,0.026250,0.315099,0.414378,0.270524,12.0



=== POINT-RELATED COLUMNS ===
['total_points_scored', 'total_def_points', 'total_def_points_scored', 'defensive_two_point_attempt', 'defensive_two_point_conv', 'defensive_extra_point_attempt', 'defensive_extra_point_conv', 'average_defensive_two_point_attempt', 'average_defensive_two_point_conv', 'average_defensive_extra_point_attempt', 'average_defensive_extra_point_conv', 'total_points_allowed']

=== TOP MISSING VALUES ===


team_x                              0
defense_snaps                       0
tackle_with_assist                  0
sack                                0
qb_hit                              0
total_def_points_scored             0
defensive_two_point_attempt         0
defensive_two_point_conv            0
defensive_extra_point_attempt       0
defensive_extra_point_conv          0
rush_snaps_def                      0
solo_tackle                         0
pass_snaps_def                      0
average_safety_def                  0
average_interception_def            0
average_fumble_def                  0
average_fumble_lost_def             0
average_fumble_forced_def           0
average_fumble_not_forced_def       0
average_fumble_out_of_bounds_def    0
dtype: int64